# Import modules

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.create_model_pickle import *

In [5]:
who

Counter	 MyModel	 get_data	 module_path	 np	 os	 pd	 pickle	 plt	 
random	 sys	 


# Import data

In [48]:
df = pd.read_json('../data/data.json')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14337 entries, 0 to 14336
Data columns (total 44 columns):
acct_type             14337 non-null object
approx_payout_date    14337 non-null int64
body_length           14337 non-null int64
channels              14337 non-null int64
country               14256 non-null object
currency              14337 non-null object
delivery_method       14321 non-null float64
description           14337 non-null object
email_domain          14337 non-null object
event_created         14337 non-null int64
event_end             14337 non-null int64
event_published       14238 non-null float64
event_start           14337 non-null int64
fb_published          14337 non-null int64
gts                   14337 non-null float64
has_analytics         14337 non-null int64
has_header            8928 non-null float64
has_logo              14337 non-null int64
listed                14337 non-null object
name                  14337 non-null object
name_length      

In [50]:

date_cols = ['approx_payout_date', 'event_created', 'event_created', 'event_end', \
             'event_published', 'event_start', 'user_created']
for col in date_cols:
    df[col] = pd.to_datetime(df[col])
    

In [51]:
df.head()

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1970-01-01 00:00:01.266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1970-01-01 00:00:01.262739706,...,"[{u'event_id': 527017, u'cost': 25.0, u'availa...",36,1970-01-01 00:00:01.259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1970-01-01 00:00:01.296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1970-01-01 00:00:01.293832670,...,"[{u'event_id': 786878, u'cost': 35.0, u'availa...",149,1970-01-01 00:00:01.280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,1970-01-01 00:00:01.296172800,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1970-01-01 00:00:01.291090956,...,"[{u'event_id': 787337, u'cost': 93.51, u'avail...",214,1970-01-01 00:00:01.272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,1970-01-01 00:00:01.388966400,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1970-01-01 00:00:01.360681570,...,"[{u'event_id': 885645, u'cost': 25.0, u'availa...",889,1970-01-01 00:00:01.283870102,3,,None,NaN,NaN,None,None
4,premium,1970-01-01 00:00:01.297900800,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1970-01-01 00:00:01.291994666,...,"[{u'event_id': 1114349, u'cost': 150.0, u'avai...",35,1970-01-01 00:00:01.288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA
